In [4]:
import numpy as np
import pandas as pd
import requests

In [5]:
FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2smeia_api_key = ''

In [11]:
api_url = 'https://api.eia.gov/v2/electricity/rto/region-data/data/?api_key=FVtyMiv9TXSjy0x4O9xmVYE2iksnAlYeEXPaD2sm&frequency=hourly&data[0]=value&start=2022-10-01T00&end=2023-10-01T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'

headers = {
    "X-Params": '{"frequency": "hourly", "data": ["value"], "facets": {}, "start": "2022-10-01T00", "end": "2023-10-01T00", "sort": [{"column": "period", "direction": "desc"}], "offset": 0, "length": 5000}'
}

response = requests.get(api_url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Now you can work with the data as needed
    # For example, to print the first 10 records:
    for record in data["data"]:
        print(record)

else:
    print(f"Error: {response.status_code} - {response.text}")

{'response': {'warnings': [{'warning': 'incomplete return', 'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}], 'total': 2502515, 'dateFormat': 'YYYY-MM-DD"T"HH24', 'frequency': 'hourly', 'data': [{'period': '2023-10-01T00', 'respondent': 'WAUW', 'respondent-name': 'Western Area Power Administration - Upper Great Plains West', 'type': 'NG', 'type-name': 'Net generation', 'value': 70, 'value-units': 'megawatthours'}, {'period': '2023-10-01T00', 'respondent': 'DUK', 'respondent-name': 'Duke Energy Carolinas', 'type': 'DF', 'type-name': 'Day-ahead demand forecast', 'value': 13091, 'value-units': 'megawatthours'}, {'period': '2023-10-01T00', 'respondent': 'TAL', 'respondent-name': 'City of Tallahassee', 'type': 'DF', 'type-name': 'Day-ahead demand forecast', 'value': 390, 'value-units': 'megawatthours'}, {'period': '2023-10-01T00', 'respondent': 'SC', 'respondent-nam